In [ ]:
#!pip install konlpy
#!pip install --upgrade wandb

In [1]:
import os
import pandas as pd
import tensorflow
import numpy as np
import transformers
import torch
import pandas as pd
import tensorflow as tf

from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

print(transformers.__version__)
print(tf.__version__)



4.51.3
2.18.0


In [ ]:
import tensorflow as tf
import os

if 'COLAB_TPU_ADDR' in os.environ:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(
        tpu='grpc://' + os.environ['COLAB_TPU_ADDR']
    )
    tf.config.experimental_connect_to_cluster(resolver)
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.TPUStrategy(resolver)
    print("TPU initialized.")
else:
    print("TPU 환경이 아님. GPU 또는 CPU 사용 중입니다.")

TPU 환경이 아님. GPU 또는 CPU 사용 중입니다.


**데이타**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC

/content/drive/MyDrive/Colab Notebooks/Aiffel/AIFFEL_DLThon_DKTC


train 데이타와 eda, synonym 증강 데이타 사용한다.

In [7]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, 'data')
train_data_path = os.path.join(data_dir, 'merged_train.csv')

train_data = pd.read_csv(train_data_path)[["conversation", "class"]]
train_data.head()

,conversation,class
0,어이 가방 좋은데!?\n네!? 왜 그러시죠\n아니 우리가 밥을 굶어서 편의점 라면이...,갈취 대화
1,요즘 읽고 있는 책 있어?\n그거 좋은 생각이야.\n좋은 시간 보냈네.\n조언 고마...,일반 대화
2,간도 커 어딜 튀려고 !\n아니 미주엄마 내가 딱 일주일만 더 쓰고 주려고 했는데 ...,협박 대화
3,야 너 뭐하냐\n공부해\n뭔공부\n토익공부\n그런다고 점수 올릴 수 있겠니?\n왜 ...,기타 괴롭힘 대화
4,아저씨 제발 살려주세요. 칼만 좀 내려놔주시면.\n소리내지마. 소리 내면 찌른다.\...,협박 대화


In [ ]:
eda_data_path = os.path.join(data_dir, 'eda_only.csv')

eda_data = pd.read_csv(eda_data_path)[["conversation", "class"]]
eda_data.head()

,conversation,class
0,팀장님 아니거 언제까지 마무리 하면 될까마무리?이무리하지됐고말고 넉넉하게 주말까지아...,직장 내 괴롭힘 대화
1,내일 로 것? 온다던데그래야겠다 우산?챙겨가야 어떻대 같아. 에이? 야외 . 계획했...,일반 대화
2,존재 쟤 에 봐. 따고꼴좀하고유행하는그만해줘 못 호박에 입었네긋는다고 되나 으엑. ...,기타 괴롭힘 대화
3,오늘 공교육 내용 이해했어? 솔직히 좀 어려웠어. 너는? 나도 몇 부분이 헷갈리더라...,일반 대화
4,자네 내일 아침에 우리집 들려서 출근하게 네? 내가 차가 고장났어 아. 그런데 실장...,직장 내 괴롭힘 대화


In [ ]:
synm_data_path = os.path.join(data_dir, 'aug_synonym_train_only.csv')

synm_data = pd.read_csv(synm_data_path)[["conversation", "class"]]
synm_data.head()

,conversation,class
0,그게 사실이야 죄송합니다 그게 솔직히 너가 그랬을 거라고 생각하지 않아 아닙니다 한...,협박 대화
1,박사원 몇살이에요 예 몇살이길래 이거 하나 제대로 못해와 혹시 학벌 속였어요 아닙니...,직장 내 괴롭힘 대화
2,아 너 진짜 싫다 왜 몰라서 물어 너희 부모님이 부탁해서 너 챙겨주긴하는데 솔직히 ...,기타 괴롭힘 대화
3,님 그 호텔 좋아요 신고했어요 신고 안 될텐데 나 아이피추적도 안 돼요 왜 이러세요...,기타 괴롭힘 대화
4,저기 과장님 회식 자리에서 그때 죄송합니다 제가 술에 너무 취해서 시끄러워 어디서 ...,직장 내 괴롭힘 대화


데이터 전처리

In [ ]:
# sequence length check
merged_data = pd.concat([train_data, eda_data], ignore_index=True)
merged_data = pd.concat([merged_data, synm_data], ignore_index=True)

In [ ]:
print('train 샘플수 :', (len(train_data)))
print('eda 샘플수 :', (len(eda_data)))
print('synm 샘플수 :', (len(synm_data)))
print('전체 샘플수 :', (len(merged_data)))
merged_data.info()

train 샘플수 : 4637
eda 샘플수 : 4637
synm 샘플수 : 8005
전체 샘플수 : 17279
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17279 entries, 0 to 17278
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   conversation  17279 non-null  object
 1   class         17279 non-null  object
dtypes: object(2)
memory usage: 270.1+ KB


In [ ]:
import re

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣ㅏ-ㅣ?.!,]+", " ", sentence)

    sentence = sentence.strip()

    return sentence

In [ ]:
merged_data["conversation"] = [preprocess_sentence(s) for s in merged_data["conversation"]]


train_data["conversation"] = [preprocess_sentence(s) for s in train_data["conversation"]]
eda_data["conversation"] = [preprocess_sentence(s) for s in eda_data["conversation"]]
synm_data["conversation"] = [preprocess_sentence(s) for s in synm_data["conversation"]]

merged_data.head()

,conversation,class
0,팀장님 이거 언제까지 마무리 하면 될까요 ? 무리하지 말고 넉넉하게 주말까지 다 작...,직장 내 괴롭힘 대화
1,"내일 날씨 어떻대 ? 비 온다던데 . 우산 챙겨가야 할 것 같아 . 에이 , 야외 ...",일반 대화
2,야 쟤 좀 봐 . 꼴에 유행하는 옷 입었네 호박에 줄 긋는다고 수박되나 으엑 . 못...,기타 괴롭힘 대화
3,오늘 수업 내용 이해했어 ? 솔직히 좀 어려웠어 . 너는 ? 나도 몇 부분이 헷갈리...,일반 대화
4,자네 내일 아침에 우리집 들려서 출근하게 네 ? 내가 차가 고장났어 아 . 그런데 ...,직장 내 괴롭힘 대화


In [ ]:
# 한 문장의 길이를  num_tokens에 저장
num_tokens = [len(tokens) for tokens in merged_data['conversation']]
# array로 변환
num_tokens = np.array(num_tokens)

# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다.
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('train_sequences maxlen : ', maxlen)
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 maxlen 설정값 이내에 포함됩니다. ')

sequences_maxlen = maxlen

sequences_maxlen = 300


문장길이 평균 :  230.59361074136234
문장길이 최대 :  1039
문장길이 표준편차 :  111.52507663568908
train_sequences maxlen :  453
전체 문장의 0.9519069390589733%가 maxlen 설정값 이내에 포함됩니다. 


In [ ]:
# 클래스 라벨을 숫자로 변환
# 0, 1, 2, 3, 4
CLASS_NAMES = ['협박 대화', '갈취 대화', '직장 내 괴롭힘 대화', '기타 괴롭힘 대화', '일반 대화']


class_dict = {}
for idx, class_name in enumerate(CLASS_NAMES):
    class_dict[class_name] = idx



train_data['label'] = train_data['class'].apply(lambda s: class_dict[s])

eda_data['label'] = eda_data['class'].apply(lambda s: class_dict[s])
synm_data['label'] = synm_data['class'].apply(lambda s: class_dict[s])

train_data.head()

,conversation,class,label
0,팀장님 이거 언제까지 마무리 하면 될까요 ? 무리하지 말고 넉넉하게 주말까지 다 작...,직장 내 괴롭힘 대화,2
1,"내일 날씨 어떻대 ? 비 온다던데 . 우산 챙겨가야 할 것 같아 . 에이 , 야외 ...",일반 대화,4
2,야 쟤 좀 봐 . 꼴에 유행하는 옷 입었네 호박에 줄 긋는다고 수박되나 으엑 . 못...,기타 괴롭힘 대화,3
3,오늘 수업 내용 이해했어 ? 솔직히 좀 어려웠어 . 너는 ? 나도 몇 부분이 헷갈리...,일반 대화,4
4,자네 내일 아침에 우리집 들려서 출근하게 네 ? 내가 차가 고장났어 아 . 그런데 ...,직장 내 괴롭힘 대화,2


**test data**

In [10]:
data_dir = os.path.join(current_dir, 'data')
test_data_path = os.path.join(data_dir, 'test.csv')


test_data = pd.read_csv(test_data_path)
test_data.head()

,idx,text
0,t_000,아가씨 담배한갑주소 네 4500원입니다 어 네 지갑어디갔지 에이 버스에서 잃어버렸나...
1,t_001,우리팀에서 다른팀으로 갈 사람 없나? 그럼 영지씨가 가는건 어때? 네? 제가요? ...
2,t_002,너 오늘 그게 뭐야 네 제가 뭘 잘못했나요.? 제대로 좀 하지 네 똑바로 좀 하지 ...
3,t_003,이거 들어바 와 이 노래 진짜 좋다 그치 요즘 이 것만 들어 진짜 너무 좋다 내가 ...
4,t_004,아무튼 앞으로 니가 내 와이파이야. .응 와이파이 온. 켰어. 반말? 주인님이라고도...


In [ ]:
# 한 문장의 길이를  num_tokens에 저장
num_tokens = [len(tokens) for tokens in test_data['text']]
# array로 변환
num_tokens = np.array(num_tokens)

# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다.
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

max_tokens = np.mean(num_tokens) + 2.5 * np.std(num_tokens)
maxlen = int(max_tokens)
print('test_sequences maxlen : ', maxlen)
print(f'전체 문장의 {np.sum(num_tokens < max_tokens) / len(num_tokens)}%가 maxlen 설정값 이내에 포함됩니다. ')

test_maxlen = maxlen

문장길이 평균 :  213.626
문장길이 최대 :  722
문장길이 표준편차 :  101.13918194250931
test_sequences maxlen :  466
전체 문장의 0.972%가 maxlen 설정값 이내에 포함됩니다. 


훈련/검증 데이터셋 분리
- validation set은 original_train_set으로 부터 생성하도록 고정
- random_state=42

In [ ]:
# train/ val 분리
random_state=42

df = train_data

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['conversation'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=random_state,
    stratify=df['label']
)


train 과 eda, synm 병합

In [ ]:

# 3. train_texts와 eda_data['conversation'] 합치기
train_texts += eda_data['conversation'].tolist()
train_texts += synm_data['conversation'].tolist()

# 4. train_labels와 eda_data['label'] 합치기
train_labels += eda_data['label'].tolist()
train_labels += synm_data['label'].tolist()

print("훈련 샘플 수:", len(train_texts))
print("검증 샘플 수:", len(val_texts))

훈련 샘플 수: 16351
검증 샘플 수: 928


**beomi/kcbert-base**

In [ ]:
from transformers import TFBertForSequenceClassification

model_name = "beomi/kcbert-base"

# 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained(model_name)

#PyTorch로 학습된 beomi/kcbert-base 모델을 TensorFlow로 변환하여 사용
from transformers import TFBertModel

model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=len(class_dict),  from_pt=True)#  클래스 수에 맞게

# tokenizer의 vocab 사이즈 확인
tokenizer.vocab_size


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


30000

In [ ]:
# TensorFlow 데이터셋으로 변환
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=sequences_maxlen)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=sequences_maxlen)

# TensorFlow 데이터셋으로 변환
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

# 배치 처리 및 셔플
train_dataset = train_dataset.shuffle(1000).batch(16)
val_dataset = val_dataset.shuffle(1000).batch(16)

In [ ]:
from transformers import AutoConfig


config = AutoConfig.from_pretrained(model_name)
config

In [ ]:
# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])



In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108918528 
                                                                 
 dropout_37 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
Total params: 108922373 (415.51 MB)
Trainable params: 108922373 (415.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
import wandb
from wandb.integration.keras.callbacks import WandbMetricsLogger
wandb.login()

# W&B 프로젝트 초기화
run = wandb.init(
    # Set the wandb entity where your project will be logged (generally your team name).
    entity="jiwoong-team",
    # Set the wandb project where this run will be logged.
    project="DLThon-DKTC",
    # Track hyperparameters and run metadata.
    name=f"moya_KcBert300_trainedasynm", # 실험 이름
    notes="train+eda+synm 데이터를 사용한 기준선 실험", # 실험에 대한 간단한 설명
    config={ # 세부 구성 내용
        "experiment_name": "kcbert",
        "general_conversation_type": "LLM으로 생성한 일반데이타", # 일반 데이터 타입
        "architecture": "kcbert-base)",
    },
)



wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: moya9292 (jiwoong-team) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# 모델 학습
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=3
    #callbacks=[WandbMetricsLogger()]  # WandbMetricsLogger를 콜백으로 넘겨주기만 하면 알아서 epoch별 metric을 기록
)

Epoch 1/3
1022/1022 [==============================] - 1102s 1s/step - loss: 0.2329 - accuracy: 0.9220 - val_loss: 0.0476 - val_accuracy: 0.9860
Epoch 2/3
1022/1022 [==============================] - 1055s 1s/step - loss: 0.0464 - accuracy: 0.9854 - val_loss: 0.0281 - val_accuracy: 0.9925
Epoch 3/3
1022/1022 [==============================] - 1057s 1s/step - loss: 0.0271 - accuracy: 0.9915 - val_loss: 0.0509 - val_accuracy: 0.9871


In [ ]:
from sklearn.metrics import classification_report
# 최적의 모델로 검증세트 예측


# 예측 수행
logits = model.predict(val_dataset).logits
# 소프트맥스 후 argmax로 클래스 인덱스 추출
y_pred_labels = np.argmax(logits, axis=-1)


# val_dataset에서 정답 라벨만 추출
y_true_labels = []

for batch in val_dataset:
    _, labels = batch  # unpack (inputs, labels)
    y_true_labels.extend(labels.numpy())

y_true_labels = np.array(y_true_labels)


# wandb Table 생성
report = classification_report(y_true_labels, y_pred_labels, target_names=CLASS_NAMES, output_dict=True)
columns = ["class", "precision", "recall", "f1-score", "support"]
data = []

for label, metrics in report.items():
    if isinstance(metrics, dict):  # dict일 때만 values 가져오기
        row = [label] + [metrics.get(col, None) for col in columns[1:]]
        data.append(row)

table = wandb.Table(columns=columns, data=data)

# wandb에 훈련 결과 기록
wandb.log({
    "classification_report_table": table,
    "macro_f1": report["macro avg"]["f1-score"],
    "accuracy": report["accuracy"],
    "threat_f1": report["협박 대화"]["f1-score"],
    "extortion_f1": report["갈취 대화"]["f1-score"],
    "workplace_bullying_f1": report["직장 내 괴롭힘 대화"]["f1-score"],
    "other bullying": report["기타 괴롭힘 대화"]["f1-score"],
    "general_conversation_f1": report["일반 대화"]["f1-score"],
})

# 실험 종료
run.finish()

58/58 [==============================] - 26s 373ms/step


accuracy,▁
extortion_f1,▁
general_conversation_f1,▁
macro_f1,▁
other bullying,▁
threat_f1,▁
workplace_bullying_f1,▁
accuracy,0.2069
extortion_f1,0.25641
general_conversation_f1,0.21587
macro_f1,0.20712


Test 데이타 예측

In [ ]:

def predict_labels(model, tokenizer, texts, max_length=128, batch_size=16):
    """
    주어진 텍스트 리스트에 대해 예측된 클래스 인덱스를 반환하는 함수.

    Args:
        model: 학습된 TensorFlow 모델
        tokenizer: Hugging Face tokenizer
        texts (List[str]): 예측할 텍스트 목록
        max_length (int): 입력 시퀀스 최대 길이
        batch_size (int): 배치 사이즈

    Returns:
        np.ndarray: 예측된 클래스 인덱스 배열
    """
    # 텍스트 인코딩
    encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)

    # TensorFlow Dataset 생성
    dataset = tf.data.Dataset.from_tensor_slices(dict(encodings)).batch(batch_size)

    # 예측 수행
    logits = model.predict(dataset).logits

    # 소프트맥스 후 argmax로 클래스 인덱스 추출
    predictions = np.argmax(logits, axis=-1)

    return predictions


In [ ]:
# 예측
test_texts = test_data["text"].tolist()
predictions = predict_labels(model, tokenizer, test_texts, max_length=sequences_maxlen)
print(predictions)

32/32 [==============================] - 14s 336ms/step
[1 2 2 3 3 0 0 1 2 1 0 3 2 2 2 0 3 2 0 3 0 3 3 1 1 2 3 2 2 1 1 1 3 1 0 3 1
 1 0 1 0 1 2 1 3 2 0 3 0 0 1 3 1 2 2 3 3 3 3 3 1 3 0 3 2 0 3 3 4 2 2 2 3 2
 0 2 2 1 2 3 1 0 4 1 3 2 1 3 1 2 2 2 0 1 3 2 1 2 1 2 1 1 3 0 2 2 2 3 3 0 3
 0 0 2 1 1 1 2 2 1 2 2 1 1 1 1 2 2 1 3 3 3 2 3 1 0 0 0 3 2 3 3 1 3 2 0 0 0
 0 1 0 3 4 1 2 3 0 2 3 3 0 3 2 0 2 3 2 2 3 1 0 2 2 3 2 3 2 2 1 1 2 3 2 2 0
 3 2 0 1 0 0 3 1 3 1 3 0 1 0 3 2 0 3 2 0 3 2 1 2 2 0 1 0 2 1 3 3 0 0 0 1 4
 0 4 0 1 1 0 1 0 2 0 3 3 1 3 3 1 0 2 3 3 3 2 3 1 2 1 2 1 2 2 0 2 2 1 3 2 1
 2 1 2 1 3 2 3 3 3 2 1 2 0 0 2 1 2 2 3 2 2 1 0 3 2 2 2 1 2 0 3 1 1 3 3 2 0
 3 3 1 0 2 3 0 2 1 1 3 2 1 2 1 1 3 3 2 3 1 3 3 0 2 0 0 1 1 1 1 1 2 3 1 0 0
 3 1 2 3 1 2 3 0 2 0 0 1 2 2 1 0 0 0 3 1 1 1 2 3 3 0 1 3 1 1 1 2 2 1 2 0 1
 1 2 0 2 2 1 1 1 1 1 2 0 2 2 3 1 0 0 0 1 1 0 1 2 1 3 0 1 1 0 3 3 2 2 3 3 2
 1 2 1 0 3 0 2 2 4 3 1 1 0 1 0 0 3 3 1 1 2 2 0 3 3 1 2 1 3 0 2 3 2 3 0 2 1
 1 2 0 2 2 2 1 0 2 3 2 1 2 2 1 1 0 1 2 2 2 3

In [5]:
# 임의 저장
#predictions = np.array([1, 2, 2, 3, 3, 0, 0, 1, 2, 1, 0, 3, 2, 2, 2, 0,
                        3, 2, 0, 3, 0, 3, 3, 1, 1, 2, 3, 2, 2, 1, 1, 1,
                        3, 1, 0, 3, 1, 1, 0, 1, 0, 1, 2, 1, 3, 2, 0, 3,
                        0, 0, 1, 3, 1, 2, 2, 3, 3, 3, 3, 3, 1, 3, 0, 3,
                        2, 0, 3, 3, 4, 2, 2, 2, 3, 2, 0, 2, 2, 1, 2, 3,
                        1, 0, 4, 1, 3, 2, 1, 3, 1, 2, 2, 2, 0, 1, 3, 2,
                        1, 2, 1, 2, 1, 1, 3, 0, 2, 2, 2, 3, 3, 0, 3, 0,
                        0, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 2, 2,
                        1, 3, 3, 3, 2, 3, 1, 0, 0, 0, 3, 2, 3, 3, 1, 3,
                        2, 0, 0, 0, 0, 1, 0, 3, 4, 1, 2, 3, 0, 2, 3, 3,
                        0, 3, 2, 0, 2, 3, 2, 2, 3, 1, 0, 2, 2, 3, 2, 3,
                        2, 2, 1, 1, 2, 3, 2, 2, 0, 3, 2, 0, 1, 0, 0, 3,
                        1, 3, 1, 3, 0, 1, 0, 3, 2, 0, 3, 2, 0, 3, 2, 1,
                        2, 2, 0, 1, 0, 2, 1, 3, 3, 0, 0, 0, 1, 4, 0, 4,
                        0, 1, 1, 0, 1, 0, 2, 0, 3, 3, 1, 3, 3, 1, 0, 2,
                        3, 3, 3, 2, 3, 1, 2, 1, 2, 1, 2, 2, 0, 2, 2, 1,
                        3, 2, 1, 2, 1, 2, 1, 3, 2, 3, 3, 3, 2, 1, 2, 0,
                        0, 2, 1, 2, 2, 3, 2, 2, 1, 0, 3, 2, 2, 2, 1, 2,
                        0, 3, 1, 1, 3, 3, 2, 0, 3, 3, 1, 0, 2, 3, 0, 2,
                        1, 1, 3, 2, 1, 2, 1, 1, 3, 3, 2, 3, 1, 3, 3, 0,
                        2, 0, 0, 1, 1, 1, 1, 1, 2, 3, 1, 0, 0, 3, 1, 2,
                        3, 1, 2, 3, 0, 2, 0, 0, 1, 2, 2, 1, 0, 0, 0, 3,
                        1, 1, 1, 2, 3, 3, 0, 1, 3, 1, 1, 1, 2, 2, 1, 2,
                        0, 1, 1, 2, 0, 2, 2, 1, 1, 1, 1, 1, 2, 0, 2, 2,
                        3, 1, 0, 0, 0, 1, 1, 0, 1, 2, 1, 3, 0, 1, 1, 0,
                        3, 3, 2, 2, 3, 3, 2, 1, 2, 1, 0, 3, 0, 2, 2, 4,
                        3, 1, 1, 0, 1, 0, 0, 3, 3, 1, 1, 2, 2, 0, 3, 3,
                        1, 2, 1, 3, 0, 2, 3, 2, 3, 0, 2, 1, 1, 2, 0, 2,
                        2, 2, 1, 0, 2, 3, 2, 1, 2, 2, 1, 1, 0, 1, 2, 2,
                        2, 3, 3, 2, 4, 0, 2, 1, 0, 3, 2, 2, 1, 3, 4, 2,
                        4, 3, 1, 0, 1, 3, 3, 2, 4, 3, 3, 4, 0, 1, 1, 2,
                        1, 1, 2, 0])


In [11]:
# 예시: id와 예측된 클래스 저장
submission_df = pd.DataFrame({
    'idx': test_data['idx'],
    'class': predictions
})

display(submission_df)

# submission.csv로 저장
submission_df.to_csv('submission_kcbert_trainedasynm.csv', index=False)

,idx,class
0,t_000,1
1,t_001,2
2,t_002,2
3,t_003,3
4,t_004,3
...,...,...
495,t_495,2
496,t_496,1
497,t_497,1
498,t_498,2
